In [1]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
!pip install folium
import json
print('numpy, pandas, ..., imported...')

numpy, pandas, ..., imported...


In [2]:
!pip install lxml


In [3]:
import requests
print('requests imported...')


from pandas.io.json import json_normalize
print('json_normalize imported...')


import matplotlib.cm as cm
import matplotlib.colors as colors
print('matplotlib imported...')


from sklearn.cluster import KMeans
print('Kmeans imported...')

!pip -q install geopy

print('geopy installed...')

!pip -q install geocoder
import geocoder

import time

requests imported...
json_normalize imported...
matplotlib imported...
Kmeans imported...
geopy installed...


In [4]:
!pip -q install geopy

print('geopy installed...')

from geopy.geocoders import Nominatim
print('Nominatim imported...')

geopy installed...
Nominatim imported...


In [5]:
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0'}
wikipedia_page = requests.get(wikipedia_link, headers = headers)
wikipedia_page

<Response [200]>

In [6]:
from bs4 import BeautifulSoup 

In [7]:
soup = BeautifulSoup(wikipedia_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody

In [8]:
rows = table.find_all('tr')

In [9]:

columns = [i.text.replace('\n', '')
           for i in rows[0].find_all('th')]



In [10]:
df = pd.DataFrame(columns = columns)
df

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref


In [11]:

for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df = df.append(pd.Series(values, index = columns), ignore_index = True)

        df

In [12]:
df.head(5)

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [13]:
df = df.rename(index=str, columns = {'Location': 'Location', 'London\xa0borough': 'Borough', 'Post town': 'Post-town', 'Postcode\xa0district': 'Postcode', 'Dial\xa0code': 'Dial-code', 'OS grid ref': 'OSGridRef'})

In [14]:
df.head(5)

,Location,Borough,Post-town,Postcode,Dial-code,OSGridRef
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [15]:
df['Borough'] = df['Borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))

In [16]:
df.shape

(531, 6)

In [17]:
df.head(5)

,Location,Borough,Post-town,Postcode,Dial-code,OSGridRef
0,Abbey Wood,"Bexley, Greenwich",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon,CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon,CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [18]:
df0 = df.drop('Postcode', axis=1).join(df['Postcode'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Postcode'))

In [19]:
df0.head(5)

,Location,Borough,Post-town,Dial-code,OSGridRef,Postcode
0,Abbey Wood,"Bexley, Greenwich",LONDON,020,TQ465785,SE2
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,020,TQ205805,W3
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,020,TQ205805,W4
10,Angel,Islington,LONDON,020,TQ345665,EC1
10,Angel,Islington,LONDON,020,TQ345665,N1


In [20]:
df1 = df0[['Location', 'Borough', 'Postcode', 'Post-town']].reset_index(drop=True)

In [21]:
df1.head(5)

,Location,Borough,Postcode,Post-town
0,Abbey Wood,"Bexley, Greenwich",SE2,LONDON
1,Acton,"Ealing, Hammersmith and Fulham",W3,LONDON
2,Acton,"Ealing, Hammersmith and Fulham",W4,LONDON
3,Angel,Islington,EC1,LONDON
4,Angel,Islington,N1,LONDON


In [22]:
df1.shape

(636, 4)

In [23]:
df2 = df1
df21 = df2[df2['Post-town'].str.contains('LONDON')]

In [24]:
df21.head(5)

,Location,Borough,Postcode,Post-town
0,Abbey Wood,"Bexley, Greenwich",SE2,LONDON
1,Acton,"Ealing, Hammersmith and Fulham",W3,LONDON
2,Acton,"Ealing, Hammersmith and Fulham",W4,LONDON
3,Angel,Islington,EC1,LONDON
4,Angel,Islington,N1,LONDON


In [25]:
df21.shape

(380, 4)

In [26]:
df3 = df21[['Location', 'Borough', 'Postcode']].reset_index(drop=True)

In [27]:
df3.head(10)

,Location,Borough,Postcode
0,Abbey Wood,"Bexley, Greenwich",SE2
1,Acton,"Ealing, Hammersmith and Fulham",W3
2,Acton,"Ealing, Hammersmith and Fulham",W4
3,Angel,Islington,EC1
4,Angel,Islington,N1
5,Church End,Brent,NW10
6,Church End,Barnet,N3
7,Clapham,"Lambeth, Wandsworth",SW4
8,Clerkenwell,Islington,EC1
9,Colindale,Barnet,NW9


In [28]:
df_london = df3
df_london.to_csv('LondonLocations.csv', index = False)
df_london.head(5)

,Location,Borough,Postcode
0,Abbey Wood,"Bexley, Greenwich",SE2
1,Acton,"Ealing, Hammersmith and Fulham",W3
2,Acton,"Ealing, Hammersmith and Fulham",W4
3,Angel,Islington,EC1
4,Angel,Islington,N1


In [29]:
df_london.Postcode = df_london.Postcode.str.strip()
df_london.head(5)

,Location,Borough,Postcode
0,Abbey Wood,"Bexley, Greenwich",SE2
1,Acton,"Ealing, Hammersmith and Fulham",W3
2,Acton,"Ealing, Hammersmith and Fulham",W4
3,Angel,Islington,EC1
4,Angel,Islington,N1


In [30]:
df_se = df_london[df_london['Postcode'].str.startswith(('SE'))].reset_index(drop=True)
df_se.head(10)



,Location,Borough,Postcode
0,Abbey Wood,"Bexley, Greenwich",SE2
1,Crofton Park,Lewisham,SE4
2,Crossness,Bexley,SE2
3,Crystal Palace,Bromley,SE19
4,Crystal Palace,Bromley,SE20
5,Crystal Palace,Bromley,SE26
6,Denmark Hill,Southwark,SE5
7,Deptford,Lewisham,SE8
8,Dulwich,Southwark,SE21
9,East Dulwich,Southwark,SE22


In [31]:
df_se

,Location,Borough,Postcode
0,Abbey Wood,"Bexley, Greenwich",SE2
1,Crofton Park,Lewisham,SE4
2,Crossness,Bexley,SE2
3,Crystal Palace,Bromley,SE19
4,Crystal Palace,Bromley,SE20
5,Crystal Palace,Bromley,SE26
6,Denmark Hill,Southwark,SE5
7,Deptford,Lewisham,SE8
8,Dulwich,Southwark,SE21
9,East Dulwich,Southwark,SE22


In [32]:
df_se_top = df_se[df_se['Borough'].isin(['Lewisham', 'Southwark', 'Lambeth', 'Hackney', 'Croydon'])].reset_index(drop=True)

In [33]:
df_se_top.head(5)

,Location,Borough,Postcode
0,Crofton Park,Lewisham,SE4
1,Denmark Hill,Southwark,SE5
2,Deptford,Lewisham,SE8
3,Dulwich,Southwark,SE21
4,East Dulwich,Southwark,SE22


In [34]:
df_se_top.shape

(46, 3)

In [35]:
df_se.shape

(80, 3)

In [36]:

def get_latlng(arcgis_geocoder):
    
    
    lat_lng_coords = None
    
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, London, United Kingdom'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords


In [37]:
sample = get_latlng('SE2')
sample

[51.492450000000076, 0.12127000000003818]

In [38]:
start = time.time()

postal_codes = df_se_top['Postcode']    
coordinates = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]

end = time.time()
print("Time of execution: ", end - start, "seconds")

Time of execution:  26.479392051696777 seconds


In [39]:
df_se_loc = df_se_top


df_se_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_se_loc['Latitude'] = df_se_coordinates['Latitude']
df_se_loc['Longitude'] = df_se_coordinates['Longitude']

In [40]:
df_se_loc.head(5)

,Location,Borough,Postcode,Latitude,Longitude
0,Crofton Park,Lewisham,SE4,51.46268,-0.03558
1,Denmark Hill,Southwark,SE5,51.47480,-0.09313
2,Deptford,Lewisham,SE8,51.48114,-0.02467
3,Dulwich,Southwark,SE21,51.44100,-0.08897
4,East Dulwich,Southwark,SE22,51.45256,-0.07076


In [41]:
df_se_loc.to_csv('SELondonLocationsCoordinates.csv', index = False)

In [42]:
df_se_loc.shape

(46, 5)

In [43]:
se_df = df_se_loc.reset_index().drop('index', axis = 1)

In [44]:
se_df.shape

(46, 5)

In [45]:
se_df

,Location,Borough,Postcode,Latitude,Longitude
0,Crofton Park,Lewisham,SE4,51.46268,-0.03558
1,Denmark Hill,Southwark,SE5,51.47480,-0.09313
2,Deptford,Lewisham,SE8,51.48114,-0.02467
3,Dulwich,Southwark,SE21,51.44100,-0.08897
4,East Dulwich,Southwark,SE22,51.45256,-0.07076
5,Elephant and Castle,Southwark,SE1,51.49996,-0.09568
6,Elephant and Castle,Southwark,SE11,51.49084,-0.11108
7,Elephant and Castle,Southwark,SE17,51.48764,-0.09542
8,Bankside,Southwark,SE1,51.49996,-0.09568
9,Forest Hill,Lewisham,SE23,51.44122,-0.04764


In [46]:
se_df.loc[se_df['Location'] == 'Lewisham']

,Location,Borough,Postcode,Latitude,Longitude
20,Lewisham,Lewisham,SE13,51.46196,-0.00754


In [47]:
se_df.loc[19, 'Location']

'Lee'

In [48]:
lewisham_lat = se_df.loc[20, 'Latitude']
lewisham_long = se_df.loc[20, 'Longitude']
lewisham_loc = se_df.loc[20, 'Location']
lewisham_postcode = se_df.loc[20, 'Postcode']

print('The latitude and longitude values of {} with postcode {}, are {}, {}.'.format(lewisham_loc,
                                                                                         lewisham_postcode,
                                                                                         lewisham_lat,
                                                                                         lewisham_long))

The latitude and longitude values of Lewisham with postcode SE13, are 51.46196000000003, -0.007539999999949032.


In [49]:
import json
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0


client_881f74a6a96c4faea2be5d5c4b1a0999 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Gsa8wppigW_j7bYid3fefntons0qUDgx1y56ygY8X5AV',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_881f74a6a96c4faea2be5d5c4b1a0999.get_object(Bucket='capstoneproject-donotdelete-pr-jofsc93fpuhu2d',Key='Foursquare.json')['Body']


if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )




In [50]:
CLIENT_ID ='GANTKUJLQBS2ZJ5TDSEESMYBR2H0KUMZRIUNJHCQRYFLRTZC'
CLIENT_SECRET ='ROVHC2I1GYU1ENP14L3NIXKG4F4B24Z2UP11QZFA2CRLMV1K'
VERSION = '20210122' 

In [51]:
LIMIT = 100 
radius = 2000 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lewisham_lat, 
    lewisham_long, 
    radius, 
    LIMIT)


url

'https://api.foursquare.com/v2/venues/explore?&client_id=GANTKUJLQBS2ZJ5TDSEESMYBR2H0KUMZRIUNJHCQRYFLRTZC&client_secret=ROVHC2I1GYU1ENP14L3NIXKG4F4B24Z2UP11QZFA2CRLMV1K&v=20210122&ll=51.46196000000003,-0.007539999999949032&radius=2000&limit=100'

In [52]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601cf351bb60154a025cbdaf'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Lewisham Central',
  'headerFullLocation': 'Lewisham Central, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 179,
  'suggestedBounds': {'ne': {'lat': 51.47996001800005,
    'lng': 0.021296961190459426},
   'sw': {'lat': 51.44395998200002, 'lng': -0.03637696119035749}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '535823bc498ec8d8da9aad5f',
       'name': 'Street Feast Model Market',
       'location': {'address': '196 Lewisham High St',
        'crossStreet': 'entrance at Molesworth St',
        'lat': 51.46020945743763,
        'lng': -0.01

In [53]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [54]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


In [55]:
nearby_venues

,name,categories,lat,lng
0,Street Feast Model Market,Street Food Gathering,51.460209,-0.012199
1,Maggie's Kitchen,Café,51.465380,-0.011213
2,Levante Pide Restaurant,Turkish Restaurant,51.459848,-0.011476
3,Gennaro Delicatessan,Deli / Bodega,51.461765,-0.009726
4,Levante restaurant,Restaurant,51.462072,-0.009491
5,Manor House Gardens,Park,51.456686,0.004684
6,Corte,Coffee Shop,51.459776,-0.011554
7,The Sausage Man,Food Truck,51.462507,-0.010248
8,Everest Curry King,Sri Lankan Restaurant,51.466012,-0.019656
9,Blackheath Farmers' Market,Farmers Market,51.465913,0.007945


In [56]:
nearby_venues_lewisham_unique = nearby_venues['categories'].value_counts().to_frame(name='Count')

In [57]:
nearby_venues_lewisham_unique.head(5)

,Count
Pub,13
Café,9
Park,7
Gastropub,4
Garden,4


In [58]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [59]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [60]:
se_venues = getNearbyVenues(names=se_df['Location'],
                                   latitudes=se_df['Latitude'],
                                   longitudes=se_df['Longitude']
                                  )

Crofton Park
Denmark Hill
Deptford
Dulwich
East Dulwich
Elephant and Castle
Elephant and Castle
Elephant and Castle
Bankside
Forest Hill
Gipsy Hill
Gipsy Hill
Grove Park
Herne Hill
Hither Green
Honor Oak
Ladywell
Ladywell
Lambeth
Lee
Lewisham
New Cross
Newington
Newington
Nunhead
Oval
Peckham
Bellingham
Rotherhithe
Selhurst
Bermondsey
South Norwood
Southend
St Johns
Surrey Quays
Tulse Hill
Tulse Hill
Upper Norwood
Walworth
Blackheath
West Norwood
Brixton
Brockley
Camberwell
Catford
Chinbrook


In [61]:
se_venues.shape

(4223, 7)

In [62]:
len(se_venues)

4223

In [63]:
se_venues['Neighbourhood'].value_counts()
se_venues.to_csv('se_venues.csv')

In [64]:
se_venues.head(5)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Crofton Park,51.46268,-0.03558,The Orchard,51.463678,-0.035699,Gastropub
1,Crofton Park,51.46268,-0.03558,Browns Of Brockley,51.464513,-0.037346,Coffee Shop
2,Crofton Park,51.46268,-0.03558,Brockley Brewery (The Brockley Brewing Company),51.462265,-0.036132,Brewery
3,Crofton Park,51.46268,-0.03558,Brockley's Rock,51.459457,-0.033868,Fish & Chips Shop
4,Crofton Park,51.46268,-0.03558,Waterintobeer,51.463712,-0.038826,Beer Store


In [65]:
se_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Bankside,100,100,100,100,100,100
Bellingham,70,70,70,70,70,70
Bermondsey,100,100,100,100,100,100
Blackheath,91,91,91,91,91,91
Brixton,100,100,100,100,100,100
Brockley,100,100,100,100,100,100
Camberwell,100,100,100,100,100,100
Catford,70,70,70,70,70,70
Chinbrook,54,54,54,54,54,54


In [66]:
print('There are {} uniques categories.'.format(len(se_venues['Venue Category'].unique())))

There are 198 uniques categories.


In [67]:
se_venue_unique_count = se_venues['Venue Category'].value_counts().to_frame(name='Count')

In [68]:
se_venue_unique_count.head(5)

,Count
Pub,426
Coffee Shop,343
Café,268
Park,234
Grocery Store,158


In [69]:
se_venue_unique_count.describe()

,Count
count,198.000000
mean,21.328283
std,49.295425
min,1.000000
25%,4.000000
50%,7.500000
75%,16.500000
max,426.000000


In [70]:
address = 'London, United Kingdom'

geolocator = Nominatim(user_agent="ln_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [71]:
!pip install folium
import folium
map_london = folium.Map(location = [51.5073219,-0.1276474], zoom_start = 12)
map_london

In [72]:
for lat, lng, borough, loc in zip(se_df['Latitude'], 
                                  se_df['Longitude'],
                                  se_df['Borough'],
                                  se_df['Location']):
    label = '{} - {}'.format(loc, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_london)  
    
display(map_london)

In [73]:
type(se_df)

pandas.core.frame.DataFrame

In [74]:
se_onehot = pd.get_dummies(se_venues[['Venue Category']], prefix = "", prefix_sep = "")

In [75]:
se_onehot['Neighbourhood'] = se_venues['Neighbourhood']

In [76]:
fixed_columns = [se_onehot.columns[-1]] + list(se_onehot.columns[:-1])
se_onehot = se_onehot[fixed_columns]

In [77]:
se_onehot.head(5)

,Neighbourhood,African Restaurant,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Butcher,Café,Canal,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Convenience Store,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Doner Restaurant,Dumpling Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Himalayan Restaurant,Historic Site,History Museum,Hostel,Hotel,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kids Store,Lake,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nature Preserve,New American Restaurant,Nightclub,Observatory,Office,Okonomiyaki Restaurant,Outdoor Sculpture,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,RV Park,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Roof Deck,Rugby Pitch,Sake Bar,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Soccer Field,Soccer Stadium,South American Restaurant,Speakeasy,Sporting Goods Shop,Sri Lankan Restaurant,Stadium,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Tourist Information Center,Track,Track Stadium,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Windmill,Wine Bar,Wine Shop,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,Crofton Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Crofton Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Crofton Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [78]:
se_onehot.loc[se_onehot['Asian Restaurant'] != 0]

,Neighbourhood,African Restaurant,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Butcher,Café,Canal,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Convenience Store,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Doner Restaurant,Dumpling Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Himalayan Restaurant,Historic Site,History Museum,Hostel,Hotel,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kids Store,Lake,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nature Preserve,New American Restaurant,Nightclub,Observatory,Office,Okonomiyaki Restaurant,Outdoor Sculpture,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,RV Park,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Roof Deck,Rugby Pitch,Sake Bar,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Soccer Field,Soccer Stadium,South American Restaurant,Speakeasy,Sporting Goods Shop,Sri Lankan Restaurant,Stadium,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Tourist Information Center,Track,Track Stadium,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Windmill,Wine Bar,Wine Shop,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
488,Elephant and Castle,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
506,Elephant and Castle,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
788,Bankside,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [79]:
se_onehot.loc[se_onehot['African Restaurant'] != 0]

,Neighbourhood,African Restaurant,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Butcher,Café,Canal,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Convenience Store,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Doner Restaurant,Dumpling Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Himalayan Restaurant,Historic Site,History Museum,Hostel,Hotel,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kids Store,Lake,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nature Preserve,New American Restaurant,Nightclub,Observatory,Office,Okonomiyaki Restaurant,Outdoor Sculpture,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,RV Park,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Roof Deck,Rugby Pitch,Sake Bar,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Soccer Field,Soccer Stadium,South American Restaurant,Speakeasy,Sporting Goods Shop,Sri Lankan Restaurant,Stadium,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Tourist Information Center,Track,Track Stadium,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Windmill,Wine Bar,Wine Shop,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
136,Denmark Hill,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
615,Elephant and Castle,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
702,Elephant and Castle,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [80]:
se_onehot.to_csv('selondon_onehot.csv', index = False)

In [81]:
se_onehot.shape

(4223, 199)

In [82]:
se_grouped = se_onehot.groupby('Neighbourhood').mean().reset_index()
se_grouped.head()

,Neighbourhood,African Restaurant,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Butcher,Café,Canal,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Convenience Store,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Doner Restaurant,Dumpling Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Himalayan Restaurant,Historic Site,History Museum,Hostel,Hotel,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kids Store,Lake,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nature Preserve,New American Restaurant,Nightclub,Observatory,Office,Okonomiyaki Restaurant,Outdoor Sculpture,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,RV Park,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Roof Deck,Rugby Pitch,Sake Bar,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Soccer Field,Soccer Stadium,South American Restaurant,Speakeasy,Sporting Goods Shop,Sri Lankan Restaurant,Stadium,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Tourist Information Center,Track,Track Stadium,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Windmill,Wine Bar,Wine Shop,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,Bankside,0.00,0.000000,0.0,0.0,0.000000,0.01,0.02,0.0,0.02,0.0,0.0,0.0,0.050000,0.000000,0.01,0.00,0.0,0.00,0.01,0.0,0.01,0.0,0.0,0.000000,0.00,0.01,0.000000,0.01,0.030000,0.0,0.0,0.0,0.01,0.01,0.000000,0.0,0.000000,0.030000,0.120000,0.00,0.000000,0.01,0.00,0.00,0.01,0.0,0.01,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.01,0.0,0.00,0.0,0.00,0.0,0.010000,0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.01,0.0,0.0,0.0,0.010000,0.000000,0.0,0.000000,0.00,0.030000,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.000000,0.030000,0.000000,0.010000,0.0,0.0,0.000000,0.000000,0.000000,0.02,0.01,0.070000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.030000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.0,0.0,0.020000,0.01,0.0,0.01,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.040000,0.0,0.00,0.00,0.000000,0.01,0.000000,0.0,0.01,0.000000,0.0,0.000000,0.06,0.0,0.02,0.000000,0.000000,0.01,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.03,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.070000,0.0,0.0,0.0,0.010000,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.020000,0.0,0.00

In [83]:
print("Before One-hot encoding:", se_df.shape)
print("After One-hot encoding:", se_grouped.shape)

Before One-hot encoding: (46, 5)
After One-hot encoding: (40, 199)


In [84]:
se_grouped.to_csv('london_grouped.csv', index = False)

In [85]:
num_top_venues = 10 
for hood in se_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = se_grouped[se_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop = True).head(num_top_venues))
    print('\n')

----Bankside----
                   venue  freq
0            Coffee Shop  0.12
1                Theater  0.07
2                  Hotel  0.07
3         Scenic Lookout  0.06
4                 Bakery  0.05
5                    Pub  0.04
6     Italian Restaurant  0.03
7          Grocery Store  0.03
8           Cocktail Bar  0.03
9  Street Food Gathering  0.03


----Bellingham----
                  venue  freq
0         Grocery Store  0.14
1                  Park  0.09
2           Supermarket  0.09
3  Fast Food Restaurant  0.04
4           Coffee Shop  0.04
5                   Pub  0.04
6    Italian Restaurant  0.04
7        Hardware Store  0.03
8              Pharmacy  0.03
9              Bus Stop  0.03


----Bermondsey----
                   venue  freq
0            Coffee Shop  0.12
1                Theater  0.07
2                  Hotel  0.07
3         Scenic Lookout  0.06
4                 Bakery  0.05
5                    Pub  0.04
6     Italian Restaurant  0.03
7          Grocery Sto

In [86]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [87]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = se_grouped['Neighbourhood']

for ind in np.arange(se_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(se_grouped.iloc[ind, :], num_top_venues)

In [88]:
neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bankside,Coffee Shop,Theater,Hotel,Scenic Lookout,Bakery,Pub,Grocery Store,Garden,Italian Restaurant,Cocktail Bar
1,Bellingham,Grocery Store,Supermarket,Park,Italian Restaurant,Pub,Coffee Shop,Fast Food Restaurant,Bus Stop,Train Station,Hardware Store
2,Bermondsey,Coffee Shop,Theater,Hotel,Scenic Lookout,Bakery,Pub,Grocery Store,Garden,Italian Restaurant,Cocktail Bar
3,Blackheath,Pub,Coffee Shop,Park,Grocery Store,Garden,Supermarket,Bakery,Café,Italian Restaurant,Bus Stop
4,Brixton,Coffee Shop,Café,Pub,Park,Italian Restaurant,Middle Eastern Restaurant,Bar,Pizza Place,Cocktail Bar,Brewery


In [89]:
neighbourhoods_venues_sorted.to_csv('neighbourhoods_venues_sorted.csv', index = False)

In [90]:
se_grouped_clustering = se_grouped.drop('Neighbourhood', 1)

In [91]:

kclusters = 5
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(se_grouped_clustering)


kmeans.labels_[0:10]

array([2, 4, 2, 3, 0, 3, 0, 4, 1, 3], dtype=int32)

In [92]:
kmeans.labels_[0:10]

array([2, 4, 2, 3, 0, 3, 0, 4, 1, 3], dtype=int32)

In [93]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [94]:
se_merged = se_df

In [95]:
se_merged.head(3)

,Location,Borough,Postcode,Latitude,Longitude
0,Crofton Park,Lewisham,SE4,51.46268,-0.03558
1,Denmark Hill,Southwark,SE5,51.47480,-0.09313
2,Deptford,Lewisham,SE8,51.48114,-0.02467


In [96]:

se_merged_latlong = se_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on = 'Location')

In [97]:
se_merged_latlong.head(5)

,Location,Borough,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Crofton Park,Lewisham,SE4,51.46268,-0.03558,3,Pub,Coffee Shop,Café,Park,Gastropub,Bar,Italian Restaurant,Turkish Restaurant,Cocktail Bar,Pizza Place
1,Denmark Hill,Southwark,SE5,51.47480,-0.09313,0,Coffee Shop,Café,Pub,Park,Italian Restaurant,Middle Eastern Restaurant,Bar,Pizza Place,Cocktail Bar,Brewery
2,Deptford,Lewisham,SE8,51.48114,-0.02467,3,Pub,Coffee Shop,Café,Park,Bar,Gym / Fitness Center,Turkish Restaurant,Cocktail Bar,Garden,Indie Movie Theater
3,Dulwich,Southwark,SE21,51.44100,-0.08897,3,Pub,Café,Park,Bakery,Grocery Store,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Farmers Market,Brewery
4,East Dulwich,Southwark,SE22,51.45256,-0.07076,0,Café,Park,Pizza Place,Italian Restaurant,Pub,Coffee Shop,Mediterranean Restaurant,Forest,Garden Center,Platform


In [98]:
se_clusters = se_merged_latlong

In [99]:
se_clusters.loc[se_clusters['Cluster Labels'] == 0, se_clusters.columns[[1] + list(range(5, se_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Southwark,0,Coffee Shop,Café,Pub,Park,Italian Restaurant,Middle Eastern Restaurant,Bar,Pizza Place,Cocktail Bar,Brewery
4,Southwark,0,Café,Park,Pizza Place,Italian Restaurant,Pub,Coffee Shop,Mediterranean Restaurant,Forest,Garden Center,Platform
13,Lambeth,0,Coffee Shop,Pub,Pizza Place,Park,Cocktail Bar,Café,Market,Burger Joint,Restaurant,Vegetarian / Vegan Restaurant
24,Southwark,0,Pub,Coffee Shop,Café,Pizza Place,Park,Italian Restaurant,Bar,Gastropub,Cocktail Bar,Burger Joint
26,Southwark,0,Pub,Coffee Shop,Café,Pizza Place,Park,Italian Restaurant,Bar,Gastropub,Cocktail Bar,Burger Joint
28,Southwark,0,Pub,Park,Brewery,Coffee Shop,Bar,Vietnamese Restaurant,Beer Bar,Café,Food Truck,Restaurant
34,Southwark,0,Pub,Park,Brewery,Coffee Shop,Bar,Vietnamese Restaurant,Beer Bar,Café,Food Truck,Restaurant
38,Southwark,0,Coffee Shop,Café,Pub,Italian Restaurant,Park,Pizza Place,Bakery,Art Gallery,Hotel,Theater
41,Lambeth,0,Coffee Shop,Café,Pub,Park,Italian Restaurant,Middle Eastern Restaurant,Bar,Pizza Place,Cocktail Bar,Brewery
43,Southwark,0,Coffee Shop,Café,Pub,Park,Italian Restaurant,Middle Eastern Restaurant,Bar,Pizza Place,Cocktail Bar,Brewery


In [100]:
se_clusters.loc[se_clusters['Cluster Labels'] == 1, se_clusters.columns[[1] + list(range(5, se_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Lewisham,1,Grocery Store,Park,Pub,Café,Italian Restaurant,Soccer Field,Gym / Fitness Center,Coffee Shop,Supermarket,Train Station
19,Lewisham,1,Grocery Store,Park,Pub,Café,Italian Restaurant,Soccer Field,Gym / Fitness Center,Coffee Shop,Supermarket,Train Station
29,Croydon,1,Grocery Store,Pub,Park,Coffee Shop,Supermarket,Café,African Restaurant,Music Store,Sandwich Place,Caribbean Restaurant
31,Croydon,1,Grocery Store,Pub,Park,Coffee Shop,Supermarket,Café,African Restaurant,Music Store,Sandwich Place,Caribbean Restaurant
45,Lewisham,1,Grocery Store,Park,Pub,Café,Italian Restaurant,Soccer Field,Gym / Fitness Center,Coffee Shop,Supermarket,Train Station


In [101]:
se_clusters.loc[se_clusters['Cluster Labels'] == 2, se_clusters.columns[[1] + list(range(5, se_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Southwark,2,Coffee Shop,Café,Pub,Theater,Hotel,Park,Italian Restaurant,Bakery,Garden,Scenic Lookout
6,Southwark,2,Coffee Shop,Café,Pub,Theater,Hotel,Park,Italian Restaurant,Bakery,Garden,Scenic Lookout
7,Southwark,2,Coffee Shop,Café,Pub,Theater,Hotel,Park,Italian Restaurant,Bakery,Garden,Scenic Lookout
8,Southwark,2,Coffee Shop,Theater,Hotel,Scenic Lookout,Bakery,Pub,Grocery Store,Garden,Italian Restaurant,Cocktail Bar
18,Lambeth,2,Coffee Shop,Theater,Hotel,Scenic Lookout,Bakery,Pub,Grocery Store,Garden,Italian Restaurant,Cocktail Bar
22,Southwark,2,Coffee Shop,Café,Pub,Theater,Hotel,Bakery,Italian Restaurant,Park,Scenic Lookout,Pizza Place
23,Southwark,2,Coffee Shop,Café,Pub,Theater,Hotel,Bakery,Italian Restaurant,Park,Scenic Lookout,Pizza Place
30,Southwark,2,Coffee Shop,Theater,Hotel,Scenic Lookout,Bakery,Pub,Grocery Store,Garden,Italian Restaurant,Cocktail Bar


In [102]:
se_clusters.loc[se_clusters['Cluster Labels'] == 3, se_clusters.columns[[1] + list(range(5, se_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lewisham,3,Pub,Coffee Shop,Café,Park,Gastropub,Bar,Italian Restaurant,Turkish Restaurant,Cocktail Bar,Pizza Place
2,Lewisham,3,Pub,Coffee Shop,Café,Park,Bar,Gym / Fitness Center,Turkish Restaurant,Cocktail Bar,Garden,Indie Movie Theater
3,Southwark,3,Pub,Café,Park,Bakery,Grocery Store,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Farmers Market,Brewery
9,Lewisham,3,Pub,Coffee Shop,Café,Grocery Store,Park,Supermarket,Gym / Fitness Center,Italian Restaurant,Indian Restaurant,Forest
10,Lambeth,3,Pub,Coffee Shop,Café,Grocery Store,Park,Italian Restaurant,Bakery,Train Station,Pizza Place,Supermarket
11,Lambeth,3,Pub,Coffee Shop,Café,Grocery Store,Park,Italian Restaurant,Bakery,Train Station,Pizza Place,Supermarket
14,Lewisham,3,Pub,Café,Park,Gastropub,Garden,Bakery,Coffee Shop,Fish & Chips Shop,Gym / Fitness Center,Restaurant
15,Lewisham,3,Pub,Coffee Shop,Café,Grocery Store,Park,Supermarket,Gym / Fitness Center,Italian Restaurant,Indian Restaurant,Forest
16,Lewisham,3,Pub,Café,Coffee Shop,Park,Gastropub,Bar,Fish & Chips Shop,Italian Restaurant,Gym / Fitness Center,Food Truck
17,Lewisham,3,Pub,Café,Coffee Shop,Park,Gastropub,Bar,Fish & Chips Shop,Italian Restaurant,Gym / Fitness Center,Food Truck


In [103]:
se_clusters.loc[se_clusters['Cluster Labels'] == 4, se_clusters.columns[[1] + list(range(5, se_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Lewisham,4,Grocery Store,Supermarket,Park,Italian Restaurant,Pub,Coffee Shop,Fast Food Restaurant,Bus Stop,Train Station,Hardware Store
32,Lewisham,4,Grocery Store,Supermarket,Park,Italian Restaurant,Pub,Coffee Shop,Fast Food Restaurant,Bus Stop,Train Station,Hardware Store
44,Lewisham,4,Grocery Store,Supermarket,Park,Italian Restaurant,Pub,Coffee Shop,Fast Food Restaurant,Bus Stop,Train Station,Hardware Store


In [104]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(se_clusters['Latitude'], se_clusters['Longitude'], se_clusters['Location'], se_clusters['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

display(map_clusters)

In [105]:
se_clusters.loc[se_clusters['Cluster Labels'] == 0, se_clusters.columns[[1] + list(range(5, se_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Southwark,0,Coffee Shop,Café,Pub,Park,Italian Restaurant,Middle Eastern Restaurant,Bar,Pizza Place,Cocktail Bar,Brewery
4,Southwark,0,Café,Park,Pizza Place,Italian Restaurant,Pub,Coffee Shop,Mediterranean Restaurant,Forest,Garden Center,Platform
13,Lambeth,0,Coffee Shop,Pub,Pizza Place,Park,Cocktail Bar,Café,Market,Burger Joint,Restaurant,Vegetarian / Vegan Restaurant
24,Southwark,0,Pub,Coffee Shop,Café,Pizza Place,Park,Italian Restaurant,Bar,Gastropub,Cocktail Bar,Burger Joint
26,Southwark,0,Pub,Coffee Shop,Café,Pizza Place,Park,Italian Restaurant,Bar,Gastropub,Cocktail Bar,Burger Joint
28,Southwark,0,Pub,Park,Brewery,Coffee Shop,Bar,Vietnamese Restaurant,Beer Bar,Café,Food Truck,Restaurant
34,Southwark,0,Pub,Park,Brewery,Coffee Shop,Bar,Vietnamese Restaurant,Beer Bar,Café,Food Truck,Restaurant
38,Southwark,0,Coffee Shop,Café,Pub,Italian Restaurant,Park,Pizza Place,Bakery,Art Gallery,Hotel,Theater
41,Lambeth,0,Coffee Shop,Café,Pub,Park,Italian Restaurant,Middle Eastern Restaurant,Bar,Pizza Place,Cocktail Bar,Brewery
43,Southwark,0,Coffee Shop,Café,Pub,Park,Italian Restaurant,Middle Eastern Restaurant,Bar,Pizza Place,Cocktail Bar,Brewery


In [106]:
se_clusters.loc[se_clusters['Cluster Labels'] == 1, se_clusters.columns[[1] + list(range(5, se_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Lewisham,1,Grocery Store,Park,Pub,Café,Italian Restaurant,Soccer Field,Gym / Fitness Center,Coffee Shop,Supermarket,Train Station
19,Lewisham,1,Grocery Store,Park,Pub,Café,Italian Restaurant,Soccer Field,Gym / Fitness Center,Coffee Shop,Supermarket,Train Station
29,Croydon,1,Grocery Store,Pub,Park,Coffee Shop,Supermarket,Café,African Restaurant,Music Store,Sandwich Place,Caribbean Restaurant
31,Croydon,1,Grocery Store,Pub,Park,Coffee Shop,Supermarket,Café,African Restaurant,Music Store,Sandwich Place,Caribbean Restaurant
45,Lewisham,1,Grocery Store,Park,Pub,Café,Italian Restaurant,Soccer Field,Gym / Fitness Center,Coffee Shop,Supermarket,Train Station


In [107]:
se_clusters.loc[se_clusters['Cluster Labels'] == 2, se_clusters.columns[[1] + list(range(5, se_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Southwark,2,Coffee Shop,Café,Pub,Theater,Hotel,Park,Italian Restaurant,Bakery,Garden,Scenic Lookout
6,Southwark,2,Coffee Shop,Café,Pub,Theater,Hotel,Park,Italian Restaurant,Bakery,Garden,Scenic Lookout
7,Southwark,2,Coffee Shop,Café,Pub,Theater,Hotel,Park,Italian Restaurant,Bakery,Garden,Scenic Lookout
8,Southwark,2,Coffee Shop,Theater,Hotel,Scenic Lookout,Bakery,Pub,Grocery Store,Garden,Italian Restaurant,Cocktail Bar
18,Lambeth,2,Coffee Shop,Theater,Hotel,Scenic Lookout,Bakery,Pub,Grocery Store,Garden,Italian Restaurant,Cocktail Bar
22,Southwark,2,Coffee Shop,Café,Pub,Theater,Hotel,Bakery,Italian Restaurant,Park,Scenic Lookout,Pizza Place
23,Southwark,2,Coffee Shop,Café,Pub,Theater,Hotel,Bakery,Italian Restaurant,Park,Scenic Lookout,Pizza Place
30,Southwark,2,Coffee Shop,Theater,Hotel,Scenic Lookout,Bakery,Pub,Grocery Store,Garden,Italian Restaurant,Cocktail Bar


In [108]:
se_clusters.loc[se_clusters['Cluster Labels'] == 3, se_clusters.columns[[1] + list(range(5, se_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lewisham,3,Pub,Coffee Shop,Café,Park,Gastropub,Bar,Italian Restaurant,Turkish Restaurant,Cocktail Bar,Pizza Place
2,Lewisham,3,Pub,Coffee Shop,Café,Park,Bar,Gym / Fitness Center,Turkish Restaurant,Cocktail Bar,Garden,Indie Movie Theater
3,Southwark,3,Pub,Café,Park,Bakery,Grocery Store,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Farmers Market,Brewery
9,Lewisham,3,Pub,Coffee Shop,Café,Grocery Store,Park,Supermarket,Gym / Fitness Center,Italian Restaurant,Indian Restaurant,Forest
10,Lambeth,3,Pub,Coffee Shop,Café,Grocery Store,Park,Italian Restaurant,Bakery,Train Station,Pizza Place,Supermarket
11,Lambeth,3,Pub,Coffee Shop,Café,Grocery Store,Park,Italian Restaurant,Bakery,Train Station,Pizza Place,Supermarket
14,Lewisham,3,Pub,Café,Park,Gastropub,Garden,Bakery,Coffee Shop,Fish & Chips Shop,Gym / Fitness Center,Restaurant
15,Lewisham,3,Pub,Coffee Shop,Café,Grocery Store,Park,Supermarket,Gym / Fitness Center,Italian Restaurant,Indian Restaurant,Forest
16,Lewisham,3,Pub,Café,Coffee Shop,Park,Gastropub,Bar,Fish & Chips Shop,Italian Restaurant,Gym / Fitness Center,Food Truck
17,Lewisham,3,Pub,Café,Coffee Shop,Park,Gastropub,Bar,Fish & Chips Shop,Italian Restaurant,Gym / Fitness Center,Food Truck


In [109]:
se_clusters.loc[se_clusters['Cluster Labels'] == 4, se_clusters.columns[[1] + list(range(5, se_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Lewisham,4,Grocery Store,Supermarket,Park,Italian Restaurant,Pub,Coffee Shop,Fast Food Restaurant,Bus Stop,Train Station,Hardware Store
32,Lewisham,4,Grocery Store,Supermarket,Park,Italian Restaurant,Pub,Coffee Shop,Fast Food Restaurant,Bus Stop,Train Station,Hardware Store
44,Lewisham,4,Grocery Store,Supermarket,Park,Italian Restaurant,Pub,Coffee Shop,Fast Food Restaurant,Bus Stop,Train Station,Hardware Store
